## NGINX Server Configuration
Sets up reverse proxy for web apps

In [1]:
# Imports
import argparse, subprocess, shutil

from flask import Flask

from misc import misc_funcs

### Command-line arguments

In [2]:
# Create parser object
cl_parser= argparse.ArgumentParser(
    description="automated site configuration for NGINX"
)

In [3]:
# Arguments

# IP and Port of Web App to reverse proxy
cl_parser.add_argument(
    "--from_site", action="store", default="localhost:8000",
    help="IP and port of web app to reverse proxy"
)

# Site Name
cl_parser.add_argument(
    "--name", action="store", default="mysite",
    help="Site name. Saves the configuration file as <name>.conf"
)

# Server hostname
cl_parser.add_argument(
    "--server_name", action="store", default="localhost",
    help="Server hostname"
)

_StoreAction(option_strings=['--server_name'], dest='server_name', nargs=None, const=None, default='localhost', type=None, choices=None, help='Server hostname', metavar=None)

### Test Command-line arguments

In [4]:
test_args= ["--from_site", "127.0.0.1:5000", "--name", "test_nginx", "--server_name", "faaizz.com"]

### Collect supplied arguments

In [5]:
# Collect command-line arguments
install_options= cl_parser.parse_args(test_args)
install_options

Namespace(from_site='127.0.0.1:5000', name='test_nginx', server_name='faaizz.com')

### Create temporary directory and set as working directory

In [6]:
# Temporary directory location
temp_path= "../temp/nginx"

# Log
print(
    "Creating temporary directory at {0}...\n".format(os.path.abspath(temp_path))
)

# Create temporary directory
misc_funcs.create_temp_dir(temp_path)

Creating temporary directory at /home/pi/Desktop/dev_ops_automation/temp/nginx...



In [8]:
# Save working directory
prev_wd=os.path.abspath("./")

# Change working directory to new temp directory
os.chdir(temp_path)

### Setup server configuration file

In [10]:
# Path to save config file
conf_path= "./{0}".format(install_options.name)
conf_path

'./test_nginx'

In [11]:
# Template path
src_path= "../../src/misc/templates/nginx_site"

In [10]:
# Setup string replacements
reps= dict(
    server_name= install_options.server_name,
    from_site= install_options.from_site,
    # FIX
    http_upgrade= "$http_upgrade",
    host= "$host"
)

In [11]:
# Log
print(
    "Populating template..."
)

Populating template...


In [12]:
# Populate template
misc_funcs.populate_template(src_path, conf_path, reps)

### Copy config file to /etc/nginx/sites-available

In [13]:
# Copy config file to /etc/nginx/sites-available

# Destination directory
dst_dir= "/etc/nginx/sites-available"

# Log
print(
    "Copying config file {0} to {1}...\n"
    .format(os.path.abspath(conf_path), dst_dir)
)

# Copy operation
shutil.copy(conf_path, dst_dir)

Copying config file /home/pi/Desktop/dev_ops_automation/temp/nginx/test_nginx to /etc/nginx/sites-available...



PermissionError: [Errno 13] Permission denied: '/etc/nginx/sites-available/test_nginx'

### NGINX Housekeeping

In [13]:
# Create symbolic link in /etc/nginx/sites-enabled

# Log
print(
    "Attempting to create symbolic link in /etc/nginx/sites-enabled...\n"
)

# Create child process
sub_proc= subprocess.Popen(
    "sudo ln -s /etc/nginx/sites-available/{0} /etc/nginx/sites-enabled/"
    .format(install_options.name), shell=True, stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT
)

# Set timeout
try:
    (out, err)= sub_proc.communicate(timeout=5)
    print(str(out) + "\n")
except subprocess.TimeoutExpired:
    print("Failed to create symbolic link in sites-enabled folder...")

Attempting to create symbolic link in /etc/nginx/sites-enabled...

b''



In [16]:
# Reload NGINX service

# Log
print(
    "Reloading NGINX service...\n"
)

sub_proc= subprocess.Popen(
    "sudo systemctl reload nginx", shell=True,
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT
)

# Set timeout
try:
    (out, err)= sub_proc.communicate(timeout=5)
    print(str(out) + "\n")
except subprocess.TimeoutExpired:
    print("NGINX service reload failed...")

Reloading NGINX service...

b''



### Cleanup

In [18]:
# Cleanup
misc_funcs.cleanup("../nginx")

Deleting files from ../nginx...



### Start Flask Web App for Demostration

In [14]:
# CHange back to inital working directory
os.chdir(prev_wd)

In [15]:
# Start test server
!./misc/run_test_flask_app.sh

* Serving Flask app "test_flask_app.py"
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
^C
